<a href="https://colab.research.google.com/github/neuhausmatheus/vicuna/blob/main/Vicuna_unfiltered_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**You may encounter an error when installing flash-attn. I couldn't figure it out. Maybe you can.**

In [ ]:
%pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
!cd ~
!git clone https://github.com/huggingface/transformers.git && cd transformers && git checkout cae78c46 && pip install .
# Install fastchat
!pip3 install --upgrade pip
!git clone https://github.com/lm-sys/FastChat && cd FastChat && pip install -e .
%pip install einops
!mkdir checkpoints
!wget https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/flash_attn-0.2.8-cp39-cp39-linux_x86_64.whl
%pip install flash_attn-0.2.8-cp39-cp39-linux_x86_64.whl

In [ ]:
!wget https://raw.githubusercontent.com/oobabooga/text-generation-webui/main/download-model.py
!mkdir models
!wget https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/ShareGPT_V2_unfiltered_cleaned_split.json
!python download-model.py decapoda-research/llama-13b-hf

**Manually edit tokenizer_config.json to: {"bos_token": "", "eos_token": "", "model_max_length": 2048, "tokenizer_class": "LlamaTokenizer", "unk_token": ""}**

**Enter wandb api key**

In [ ]:
%pip install wandb
import wandb
wandb.login()

**8 x A100 80gb training run** 

In [ ]:
!torchrun --nnodes=1 --nproc_per_node=8 --master_port=21001 \
    FastChat/fastchat/train/train.py \
    --model_name_or_path models/decapoda-research_llama-13b-hf \
    --data_path ShareGPT_unfiltered_cleaned_split.json \
    --bf16 True \
    --output_dir ./checkpoints \
    --num_train_epochs 3 \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 1200 \
    --save_total_limit 100 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --fsdp "full_shard auto_wrap" \
    --fsdp_transformer_layer_cls_to_wrap 'LlamaDecoderLayer' \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --lazy_preprocess True